* Unsloth 2024.11.10
* PyTorch 2.5.1
* TRL 0.12.1
* Datasets 3.1.0


In [1]:
! pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of trl to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72

In [2]:
!pip install --upgrade --no-deps 'unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git'

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-7vykyh2v/unsloth_8b0b21063e724271b66e915b1bcbb92e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-7vykyh2v/unsloth_8b0b21063e724271b66e915b1bcbb92e
  Resolved https://github.com/unslothai/unsloth.git to commit 229e2ecc67756f36316dfcbea42396f59eef44e0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!pip install  --no-deps torch xformers trl peft accelerate bitsandbytes triton

In [4]:
! pip install huggingface-hub

In [5]:
from unsloth import FastLanguageModel
import torch
from transformers import TextStreamer
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import os
from google.colab import userdata
from huggingface_hub import upload_file, HfApi


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


* https://huggingface.co/datasets/emdemor/sql-create-context-pt
* https://huggingface.co/unsloth/Llama-3.1-8B

In [6]:
checkpoint_modelo = 'unsloth/Llama-3.1-8B'

In [7]:
modelo, tokenizador = FastLanguageModel.from_pretrained(
    model_name=checkpoint_modelo,
    # max_seg_lenght=2048,
    dtype=None,
    load_in_4bit=True
  )

==((====))==  Unsloth 2025.9.7: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [8]:
modelo

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((409

In [9]:
tokenizador

PreTrainedTokenizerFast(name_or_path='unsloth/llama-3.1-8b-unsloth-bnb-4bit', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'pad_token': '<|finetune_right_pad_id|>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1

In [10]:
prompt = "Mê dê uma query SQL para saber quantas pessoas tem mais de 56 anos."

In [11]:
prompt_tokenizado = tokenizador([prompt], return_tensors='pt').to('cuda')

In [12]:
prompt_tokenizado

{'input_ids': tensor([[128000,     44,   5615,    294,   5615,  10832,   3319,   8029,   3429,
          42104,  10484,    300,  47062,   1592,  10071,    409,    220,   3487,
          38101,     13]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [13]:
FastLanguageModel.for_inference(modelo)
streamer_texto = TextStreamer(tokenizador,
                              skip_prompt=True,
                              skip_special_tokens=True)
_ = modelo.generate(**prompt_tokenizado, streamer=streamer_texto, max_new_tokens=128)

In [14]:
dataset = load_dataset('emdemor/sql-create-context-pt', split='train')

README.md: 0.00B [00:00, ?B/s]

sql-pt.parquet:   0%|          | 0.00/6.61M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [15]:
dataset

Dataset({
    features: ['pergunta', 'contexto', 'resposta'],
    num_rows: 78577
})

In [16]:
dataset.to_pandas().head()

,pergunta,contexto,resposta
0,Quantos chefes de departamento têm mais de 56 ...,CREATE TABLE head (age INTEGER),SELECT COUNT(*) FROM head WHERE age > 56
1,"Indicar o nome, estado de nascimento e idade d...","CREATE TABLE head (name VARCHAR, born_state VA...","SELECT name, born_state, age FROM head ORDER B..."
2,"Indique o ano de criação, o nome e o orçamento...","CREATE TABLE department (creation VARCHAR, nam...","SELECT creation, name, budget_in_billions FROM..."
3,Qual é o orçamento máximo e mínimo dos departa...,CREATE TABLE department (budget_in_billions IN...,"SELECT MAX(budget_in_billions), MIN(budget_in_..."
4,Qual é o número médio de empregados dos depart...,CREATE TABLE department (num_employees INTEGER...,SELECT AVG(num_employees) FROM department WHER...


{{- if and .First .System }}
### Input:
{{ .System }}
{{- end }}
### Instruction:
{{ .Prompt }}
### Response:

In [17]:
def gerar_prompt_sql(contexto, pergunta, resposta = ''):
    return f'''Você é um modelo poderoso de texto-para-SQL. Seu trabalho é responder perguntas sobre um banco de dados. Você recebe uma pergunta e o contexto relacionado a uma ou mais tabelas.

Você deve gerar a consulta SQL que responde à pergunta.

### Instruction:
Contexto: {contexto}

### Input:
Pergunta: {pergunta}

### Response:
Resposta: {resposta}
'''

In [18]:
dataset[0]

{'pergunta': 'Quantos chefes de departamento têm mais de 56 anos ?',
 'contexto': 'CREATE TABLE head (age INTEGER)',
 'resposta': 'SELECT COUNT(*) FROM head WHERE age > 56'}

In [19]:
print(gerar_prompt_sql(dataset[0]['contexto'], dataset[0]['pergunta'], dataset[0]['resposta']))

Você é um modelo poderoso de texto-para-SQL. Seu trabalho é responder perguntas sobre um banco de dados. Você recebe uma pergunta e o contexto relacionado a uma ou mais tabelas.

Você deve gerar a consulta SQL que responde à pergunta.

### Instruction:
Contexto: CREATE TABLE head (age INTEGER)

### Input:
Pergunta: Quantos chefes de departamento têm mais de 56 anos ?

### Response:
Resposta: SELECT COUNT(*) FROM head WHERE age > 56



In [20]:
EOS_TOKEN = tokenizador.eos_token
EOS_TOKEN

'<|end_of_text|>'

In [21]:
def formatar_prompts(dados):
    contextos = dados['contexto']
    perguntas = dados['pergunta']
    respostas = dados['resposta']
    textos = []
    for contexto, pergunta, resposta in zip(contextos, perguntas, respostas):
        texto = gerar_prompt_sql(contexto, pergunta, resposta) + EOS_TOKEN
        textos.append(texto)
    # Aqui já tokenizamos
    tokens = tokenizador(textos, truncation=True, max_length=2048, padding='max_length')
    return {"input_ids": tokens["input_ids"], "attention_mask": tokens["attention_mask"]}


In [22]:
dataset = dataset.map(formatar_prompts, batched = True)

Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [23]:
dataset.to_pandas().head()

,pergunta,contexto,resposta,input_ids,attention_mask
0,Quantos chefes de departamento têm mais de 56 ...,CREATE TABLE head (age INTEGER),SELECT COUNT(*) FROM head WHERE age > 56,"[128000, 70386, 4046, 4543, 37042, 29638, 2824...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"Indicar o nome, estado de nascimento e idade d...","CREATE TABLE head (name VARCHAR, born_state VA...","SELECT name, born_state, age FROM head ORDER B...","[128000, 70386, 4046, 4543, 37042, 29638, 2824...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"Indique o ano de criação, o nome e o orçamento...","CREATE TABLE department (creation VARCHAR, nam...","SELECT creation, name, budget_in_billions FROM...","[128000, 70386, 4046, 4543, 37042, 29638, 2824...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Qual é o orçamento máximo e mínimo dos departa...,CREATE TABLE department (budget_in_billions IN...,"SELECT MAX(budget_in_billions), MIN(budget_in_...","[128000, 70386, 4046, 4543, 37042, 29638, 2824...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Qual é o número médio de empregados dos depart...,CREATE TABLE department (num_employees INTEGER...,SELECT AVG(num_employees) FROM department WHER...,"[128000, 70386, 4046, 4543, 37042, 29638, 2824...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## LoRA

In [24]:
modelo = FastLanguageModel.get_peft_model(
    modelo,
    r=16,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
    lora_alpha=64,
    lora_dropout=0.0,
    bias='none',
    use_gradient_checkpointing='unsloth',
    random_state=10,
    use_rslora=False,
    loftq_config=None
)

Unsloth 2025.9.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [25]:
trainer = SFTTrainer(
    model = modelo,
    tokenizer = tokenizador,
    train_dataset = dataset,
    dataset_text_field = 'texto',
    max_seq_length= 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        learning_rate = 2e-5,
        max_steps = 60,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = 'adamw_8bit',
        weight_decay = 0.01,
        lr_scheduler_type = 'linear',
        seed = 10,
        output_dir = 'outputs'
    )
)



In [26]:
FastLanguageModel.for_inference(modelo)
prompt_tokenizado = tokenizador(
    [gerar_prompt_sql(
        'CREATE TABLE head (age INTEGER)',
        'Quantos chefes de departamento têm mais de 56 anos.',
        ''
    )], return_tensors = 'pt'
).to('cuda')

streamer_texto = TextStreamer(tokenizador)

_ = modelo.generate(**prompt_tokenizado, streamer = streamer_texto, max_new_tokens= 64)

<|begin_of_text|>Você é um modelo poderoso de texto-para-SQL. Seu trabalho é responder perguntas sobre um banco de dados. Você recebe uma pergunta e o contexto relacionado a uma ou mais tabelas.

Você deve gerar a consulta SQL que responde à pergunta.

### Instruction:
Contexto: CREATE TABLE head (age INTEGER)

### Input:
Pergunta: Quantos chefes de departamento têm mais de 56 anos.

### Response:
Resposta: 
SELECT * FROM head WHERE age > 56

### Instruction:
Contexto: CREATE TABLE head (age INTEGER, name TEXT)

### Input:
Pergunta: Quem tem mais de 56 anos?

### Response:
Resposta: 
SELECT name FROM head WHERE age > 56

### Instruction:
Contexto


## GGUF

In [27]:
! huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [ ]:
modelo.push_to_hub_gguf('Comune/llama-3.1-8B-fine-tunned', tokenizador, quantization_method = 'q4_k_m')

In [37]:
upload_file(
    path_or_fileobj="/content/Comune/llama-3.1-8B-fine-tunned/unsloth.Q4_K_M.gguf",
    path_in_repo="unsloth.Q4_K_M.gguf",
    repo_id="Comune/llama-3.1-8B-fine-tunned",
    repo_type="model",
    token=userdata.get('HF_TOKEN')
)


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...-8B-fine-tunned/unsloth.Q4_K_M.gguf:   0%|          | 8.40MB / 4.92GB            

CommitInfo(commit_url='https://huggingface.co/Comune/llama-3.1-8B-fine-tunned/commit/cdcd4d382372ea885ff48105323bcfab263ce6ac', commit_message='Upload unsloth.Q4_K_M.gguf with huggingface_hub', commit_description='', oid='cdcd4d382372ea885ff48105323bcfab263ce6ac', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Comune/llama-3.1-8B-fine-tunned', endpoint='https://huggingface.co', repo_type='model', repo_id='Comune/llama-3.1-8B-fine-tunned'), pr_revision=None, pr_num=None)